In [1]:
import pandas as pd

In [2]:
!pip install propy3

In [3]:
df_raw = pd.DataFrame(columns=["sequence", "class"])      

rows = []

with open("../data/raw/anti_cp/anticp2_alternate_internal_negative.txt") as reader:
    for line in reader:
        rows.append({"sequence": line.strip("\n"), "class": 0, "subset": "train"})

with open("../data/raw/anti_cp/anticp2_alternate_internal_positive.txt") as reader:
    for line in reader:
        rows.append({"sequence": line.strip("\n"), "class": 1, "subset": "train"})

with open("../data/raw/anti_cp/anticp2_alternate_validation_negative.txt") as reader:
    for line in reader:
        rows.append({"sequence": line.strip("\n"), "class": 0, "subset": "test"})

with open("../data/raw/anti_cp/anticp2_alternate_validation_positive.txt") as reader:
    for line in reader:
        rows.append({"sequence": line.strip("\n"), "class": 1, "subset": "test"})

df = pd.DataFrame(rows)

base_X_train = df.query('subset == "train"').drop(['class', 'subset'], axis=1)
base_y_train = df.query('subset == "train"')['class']

base_X_test = df.query('subset == "test"').drop(['class', 'subset'], axis=1)
base_y_test = df.query('subset == "test"')['class']

In [4]:
import itertools

aas = 'ACDEFGHIKLMNPQRSTVWY'

def compute_di_aa_composition(sequence):
    
    di_aas = sorted([''.join(di_aa) for di_aa in set(itertools.product(aas, aas))])
    di_aas_count = {di_aa:sequence.count(di_aa) for di_aa in di_aas}
    di_aas_count = {di_aa:di_aas_count[di_aa] / sum(di_aas_count.values()) for di_aa in di_aas}
    return di_aas_count

In [5]:
import collections
 
from scipy.stats import entropy
  
def estimate_shannon_entropy(protein_sequence):
    aas = collections.Counter([tmp_base for tmp_base in protein_sequence])
    # define distribution
    dist = [x/sum(aas.values()) for x in aas.values()]
 
    # use scipy to calculate entropy
    entropy_value = entropy(dist, base=2)
 
    return entropy_value

estimate_shannon_entropy(aas)

4.321928094887362

In [6]:
df_raw['shannon'] = df_raw['sequence'].map(estimate_shannon_entropy)

In [7]:
!pip install plotly

In [9]:
from propy import PyPro
from propy.GetProteinFromUniprot import GetProteinSequence

DesObject = PyPro.GetProDes(aas)  # construct a GetProDes object
paac = DesObject.GetPAAC(lamda=10, weight=0.05)
paac

{'PAAC1': 2.469,
 'PAAC2': 2.469,
 'PAAC3': 2.469,
 'PAAC4': 2.469,
 'PAAC5': 2.469,
 'PAAC6': 2.469,
 'PAAC7': 2.469,
 'PAAC8': 2.469,
 'PAAC9': 2.469,
 'PAAC10': 2.469,
 'PAAC11': 2.469,
 'PAAC12': 2.469,
 'PAAC13': 2.469,
 'PAAC14': 2.469,
 'PAAC15': 2.469,
 'PAAC16': 2.469,
 'PAAC17': 2.469,
 'PAAC18': 2.469,
 'PAAC19': 2.469,
 'PAAC20': 2.469,
 'PAAC21': 4.884,
 'PAAC22': 5.415,
 'PAAC23': 5.069,
 'PAAC24': 6.328,
 'PAAC25': 4.612,
 'PAAC26': 3.657,
 'PAAC27': 5.797,
 'PAAC28': 4.553,
 'PAAC29': 5.104,
 'PAAC30': 5.2}

In [10]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import numpy as np

def compute_sequence_data(sequence):
    sequence = sequence.upper()
    protein_analysis = ProteinAnalysis(sequence)
    data = compute_di_aa_composition(sequence)    
    data['shannon_entropy'] = estimate_shannon_entropy(sequence)
    data['molecular_weight'] = protein_analysis.molecular_weight()
    data['aromaticity'] = protein_analysis.aromaticity()
    data['instability_index'] = protein_analysis.instability_index()
    data['gravy'] = protein_analysis.gravy()
    data['isoelectric_point'] = protein_analysis.isoelectric_point()
    data['length'] = len(sequence)
    for i in range(100):
        data['charge_at_pH_%.2f'] = protein_analysis.charge_at_pH(i/10)
    
    try:
        paac = PyPro.GetProDes(sequence.upper()).GetPAAC(lamda=5, weight=0.05)
        ctd  = PyPro.GetProDes(sequence.upper()).GetCTD()
    except:
        paac = {}
        ctd  = {}
    
    data = {**data, **paac}
    data = {**data, **ctd}
    return data

In [11]:
X_train = pd.DataFrame([compute_sequence_data(sequence) for sequence in base_X_train['sequence']]).fillna(0)
X_test  = pd.DataFrame([compute_sequence_data(sequence) for sequence in base_X_test['sequence']]).fillna(0)

In [12]:
y_train = base_y_train
y_test  = base_y_test

In [13]:
X_train

,AA,AC,AD,AE,AF,AG,AH,AI,AK,AL,...,_HydrophobicityD2001,_HydrophobicityD2025,_HydrophobicityD2050,_HydrophobicityD2075,_HydrophobicityD2100,_HydrophobicityD3001,_HydrophobicityD3025,_HydrophobicityD3050,_HydrophobicityD3075,_HydrophobicityD3100
0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,18.182,54.545,18.182,27.273,54.545,9.091,9.091,72.727,81.818,100.000
1,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,8.333,41.667,61.111,75.000,100.000,11.111,11.111,13.889,16.667,36.111
2,0.043478,0.000000,0.0,0.000000,0.0,0.000000,0.021739,0.065217,0.000000,0.021739,...,2.128,17.021,34.043,65.957,97.872,8.511,38.298,61.702,82.979,100.000
3,0.038462,0.000000,0.0,0.038462,0.0,0.000000,0.000000,0.000000,0.000000,0.038462,...,22.222,25.926,51.852,77.778,100.000,3.704,3.704,11.111,88.889,96.296
4,0.085714,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.703,24.324,64.865,81.081,100.000,8.108,13.514,29.730,51.351,89.189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,...,15.385,69.231,15.385,46.154,69.231,7.692,7.692,38.462,61.538,100.000
1548,0.058824,0.000000,0.0,0.000000,0.0,0.117647,0.000000,0.000000,0.000000,0.000000,...,5.556,16.667,50.000,72.222,100.000,33.333,33.333,38.889,55.556,88.889
1549,0.000000,0.035714,0.0,0.000000,0.0,0.035714,0.000000,0.000000,0.000000,0.000000,...,3.448,17.241,51.724,65.517,93.103,13.793,27.586,44.828,62.069,89.655
1550,0.000000,0.000000,0.0,0.000000,0.0,0.022222,0.000000,0.000000,0.022222,0.000000,...,4.348,28.261,47.826,80.435,95.652,6.522,8.696,39.130,69.565,86.957


In [14]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score 

model = SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.7345360824742269

In [15]:
recall_score(y_test, y_pred)

0.7164948453608248

In [16]:
precision_score(y_test, y_pred)

0.7433155080213903

In [17]:
f1_score(y_test, y_pred)

0.7296587926509186

In [18]:
roc_auc_score(y_test, y_pred)

0.7345360824742269

In [19]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.8685567010309279

In [20]:
recall_score(y_test, y_pred)

0.8969072164948454

In [21]:
precision_score(y_test, y_pred)

0.848780487804878

In [22]:
f1_score(y_test, y_pred)

0.8721804511278195

In [23]:
roc_auc_score(y_test, y_pred)

0.8685567010309279

In [24]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.9020618556701031

In [25]:
recall_score(y_test, y_pred)

0.8969072164948454

In [26]:
precision_score(y_test, y_pred)

0.90625

In [27]:
f1_score(y_test, y_pred)

0.9015544041450777

In [28]:
roc_auc_score(y_test, y_pred)

0.902061855670103

In [29]:
from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.9020618556701031

In [30]:
recall_score(y_test, y_pred)

0.8814432989690721

In [31]:
precision_score(y_test, y_pred)

0.9193548387096774

In [32]:
f1_score(y_test, y_pred)

0.8999999999999999

In [33]:
roc_auc_score(y_test, y_pred)

0.9020618556701032

In [34]:
from xgboost import XGBClassifier

model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.9201030927835051

In [35]:
recall_score(y_test, y_pred)

0.9072164948453608

In [36]:
precision_score(y_test, y_pred)

0.9312169312169312

In [37]:
f1_score(y_test, y_pred)

0.9190600522193212

In [38]:
roc_auc_score(y_test, y_pred)

0.9201030927835051

In [39]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.8582474226804123

In [40]:
recall_score(y_test, y_pred)

0.865979381443299

In [41]:
precision_score(y_test, y_pred)

0.8527918781725888

In [42]:
f1_score(y_test, y_pred)

0.8593350383631715

In [43]:
roc_auc_score(y_test, y_pred)

0.8582474226804123

In [44]:
from sklearn.tree import ExtraTreeClassifier

model = ExtraTreeClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.75

In [45]:
recall_score(y_test, y_pred)

0.8144329896907216

In [46]:
precision_score(y_test, y_pred)

0.7214611872146118

In [47]:
f1_score(y_test, y_pred)

0.765133171912833

In [48]:
roc_auc_score(y_test, y_pred)

0.75

In [49]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.7989690721649485

In [50]:
recall_score(y_test, y_pred)

0.7835051546391752

In [51]:
precision_score(y_test, y_pred)

0.8085106382978723

In [52]:
f1_score(y_test, y_pred)

0.7958115183246073

In [53]:
roc_auc_score(y_test, y_pred)

0.7989690721649484

## Deep learning

### Recurrent

In [54]:
from tensorflow import keras

2023-03-29 11:22:26.346821: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 11:22:26.604500: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-29 11:22:27.185157: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-29 11:22:27.185235: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [55]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level = True)
tokenizer.fit_on_texts(base_X_train["sequence"] )
X_train = keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(base_X_train['sequence']), maxlen = 50)
X_test = keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(base_X_test['sequence']), maxlen = 50)

In [142]:
import pickle

with open('../data/models/classifier_tokenizer.pickle', 'wb') as writer:
    writer.write(pickle.dumps(tokenizer))

In [57]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(base_y_train)
y_train_encode = keras.utils.to_categorical(le.transform(base_y_train))
y_test_encode = keras.utils.to_categorical(le.transform(base_y_test))

In [143]:
import pickle

with open('../data/models/label_encoder.pickle', 'wb') as writer:
    writer.write(pickle.dumps(le))

In [58]:
base_X_train

,sequence
0,LYHEKYKVVEL
1,RKAVLLEEQGIEWKPEDTARPSGPREGGRRDGGRDG
2,YAAIPLGAAIGALTSGQLAHSVRPGLIMLVSTVGSFLAVGLFAIMPV
3,LLINKSPEERAALAEERTEGGTPLLIA
4,AAVLVLIHAAVRRSDNLFLDEEAAAVTEASGLMSYPS
...,...
1547,FAKLLAKLAKLKL
1548,TTGGTCCTTCAAGAGCTG
1549,GDACGETCFTGICFTAGCSCNPWPTCTRN
1550,KSCCKNTTGRNIYNTCRFAGGSRERCAKLSGCKIISASTCPSYPDK


In [59]:
y_train_encode

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [84]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

model = keras.models.Sequential()
model.add(keras.layers.Embedding(21, 21, input_length = 50))
model.add(keras.layers.LSTM(32))
model.add(keras.layers.Dense(2))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [85]:
model.fit(X_train, y_train_encode, validation_data = (X_test, y_test_encode), shuffle = True, epochs = 50, callbacks=[callback])

Epoch 1/50
49/49 [==============================] - 2s 14ms/step - loss: 1.1119 - acc: 0.6430 - val_loss: 0.6849 - val_acc: 0.7139
Epoch 2/50
49/49 [==============================] - 0s 9ms/step - loss: 0.6691 - acc: 0.7403 - val_loss: 0.6361 - val_acc: 0.8015
Epoch 3/50
49/49 [==============================] - 0s 8ms/step - loss: 0.5381 - acc: 0.7893 - val_loss: 0.5791 - val_acc: 0.8557
Epoch 4/50
49/49 [==============================] - 0s 8ms/step - loss: 0.4853 - acc: 0.8415 - val_loss: 0.5011 - val_acc: 0.8918
Epoch 5/50
49/49 [==============================] - 0s 9ms/step - loss: 0.4425 - acc: 0.8466 - val_loss: 0.3979 - val_acc: 0.8660
Epoch 6/50
49/49 [==============================] - 0s 9ms/step - loss: 0.4220 - acc: 0.8537 - val_loss: 0.4068 - val_acc: 0.8892
Epoch 7/50
49/49 [==============================] - 0s 8ms/step - loss: 0.4285 - acc: 0.8692 - val_loss: 0.4136 - val_acc: 0.9021
Epoch 8/50
49/49 [==============================] - 0s 8ms/step - loss: 0.5511 - acc: 0.7

In [62]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score

y_pred = le.inverse_transform(np.argmax(model.predict(X_test), axis = 1))
accuracy_score(base_y_test, y_pred)


13/13 [==============================] - 0s 2ms/step


0.7448453608247423

In [63]:
y_pred = le.inverse_transform(np.argmax(model.predict(X_test), axis = 1))
recall_score(base_y_test, y_pred)

13/13 [==============================] - 0s 2ms/step


0.9381443298969072

In [64]:
y_pred = le.inverse_transform(np.argmax(model.predict(X_test), axis = 1))
precision_score(base_y_test, y_pred)

13/13 [==============================] - 0s 2ms/step


0.6765799256505576

In [65]:
y_pred = le.inverse_transform(np.argmax(model.predict(X_test), axis = 1))
f1_score(base_y_test, y_pred)

13/13 [==============================] - 0s 2ms/step


0.7861771058315334

In [66]:
y_pred = le.inverse_transform(np.argmax(model.predict(X_test), axis = 1))
roc_auc_score(base_y_test, y_pred)

13/13 [==============================] - 0s 2ms/step


0.7448453608247423

### Convolutional

In [86]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)


model = keras.models.Sequential()
model.add(keras.layers.Embedding(21, 21, input_length = 50))
model.add(keras.layers.Conv1D(64, 8))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Conv1D(32, 8))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Conv1D(16, 8))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(2))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [87]:
model.fit(X_train, y_train_encode, validation_data = (X_test, y_test_encode), shuffle = True, epochs = 50, callbacks=[callback])

Epoch 1/50
49/49 [==============================] - 1s 5ms/step - loss: 1.1994 - acc: 0.5451 - val_loss: 0.6505 - val_acc: 0.6624
Epoch 2/50
49/49 [==============================] - 0s 3ms/step - loss: 0.6977 - acc: 0.6430 - val_loss: 0.6122 - val_acc: 0.7113
Epoch 3/50
49/49 [==============================] - 0s 3ms/step - loss: 0.6273 - acc: 0.6927 - val_loss: 0.4891 - val_acc: 0.7835
Epoch 4/50
49/49 [==============================] - 0s 3ms/step - loss: 1.1430 - acc: 0.7249 - val_loss: 0.6565 - val_acc: 0.7706
Epoch 5/50
49/49 [==============================] - 0s 3ms/step - loss: 1.0568 - acc: 0.7371 - val_loss: 0.4689 - val_acc: 0.8222
Epoch 6/50
49/49 [==============================] - 0s 3ms/step - loss: 0.6060 - acc: 0.8215 - val_loss: 0.4815 - val_acc: 0.8814
Epoch 7/50
49/49 [==============================] - 0s 3ms/step - loss: 0.6389 - acc: 0.8170 - val_loss: 0.6025 - val_acc: 0.8247
Epoch 8/50
49/49 [==============================] - 0s 3ms/step - loss: 0.8629 - acc: 0.77

In [88]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score

y_pred = le.inverse_transform(np.argmax(model.predict(X_test), axis = 1))
accuracy_score(base_y_test, y_pred)

13/13 [==============================] - 0s 841us/step


0.8324742268041238

In [89]:
recall_score(base_y_test, y_pred)

0.6958762886597938

In [90]:
precision_score(base_y_test, y_pred)

0.9574468085106383

In [91]:
f1_score(base_y_test, y_pred)

0.8059701492537313

In [92]:
roc_auc_score(base_y_test, y_pred)

0.8324742268041236

### Transformer

In [93]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [95]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [138]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

vocab_size = 21
maxlen = 50

embed_dim = 20
num_heads = 2
ff_dim = 32

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)

x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)

outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [139]:
model.fit(X_train, y_train_encode, validation_data = (X_test, y_test_encode), shuffle = True, epochs = 50, callbacks=[callback])

Epoch 1/50
49/49 [==============================] - 1s 3ms/step - loss: 0.6880 - accuracy: 0.6321 - val_loss: 0.6801 - val_accuracy: 0.6830
Epoch 2/50
49/49 [==============================] - 0s 1ms/step - loss: 0.6640 - accuracy: 0.6991 - val_loss: 0.6426 - val_accuracy: 0.7139
Epoch 3/50
49/49 [==============================] - 0s 1ms/step - loss: 0.6152 - accuracy: 0.7152 - val_loss: 0.5843 - val_accuracy: 0.7191
Epoch 4/50
49/49 [==============================] - 0s 1ms/step - loss: 0.5539 - accuracy: 0.7481 - val_loss: 0.5201 - val_accuracy: 0.7706
Epoch 5/50
49/49 [==============================] - 0s 1ms/step - loss: 0.4964 - accuracy: 0.7803 - val_loss: 0.4566 - val_accuracy: 0.8222
Epoch 6/50
49/49 [==============================] - 0s 1ms/step - loss: 0.4389 - accuracy: 0.8106 - val_loss: 0.3950 - val_accuracy: 0.8454
Epoch 7/50
49/49 [==============================] - 0s 1ms/step - loss: 0.3917 - accuracy: 0.8454 - val_loss: 0.3589 - val_accuracy: 0.8582
Epoch 8/50
49/49 [==

In [140]:
model.save_weights('../data/models/transformer.h5')

In [141]:
y_train_encode

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [121]:
y_pred = np.argmax(model.predict(X_test), axis=1)

13/13 [==============================] - 0s 2ms/step


In [123]:
accuracy_score(base_y_test, y_pred)

0.904639175257732

In [124]:
recall_score(base_y_test, y_pred)

0.8608247422680413

In [125]:
precision_score(base_y_test, y_pred)

0.943502824858757

In [126]:
f1_score(base_y_test, y_pred)

0.9002695417789758

In [127]:
roc_auc_score(base_y_test, y_pred)

0.9046391752577319